In [1]:
using Revise
push!(LOAD_PATH, ".")

3-element Array{Any,1}:
 "/Applications/Julia-0.6.app/Contents/Resources/julia/local/share/julia/site/v0.6"
 "/Applications/Julia-0.6.app/Contents/Resources/julia/share/julia/site/v0.6"      
 "."                                                                               

In [73]:
using PATHSolver
import PATHInterface
reload("PATHInterface")
using Base.Test

In [74]:
# find x such that 0 <= x <= 1
lb = [0.]
ub = [1.]
f = x -> zeros(1)
options(output=:no)
status, z, F = solveLCP(f, lb, ub);
@test z ≈ [0]

Reading options file path.opt
Read of options file complete.

1 row/cols, 0 non-zeros, 0.00% dense.

Path 4.7.03 (Thu Jan 24 15:44:12 2013)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


Test Passed

In [75]:
# x == 0 or y == 0
lb = [-Inf, -Inf]
ub = [Inf, Inf]
f = z -> [z[2], z[1]]
options(output=:no)
status, z, F = solveLCP(f, lb, ub)

Reading options file path.opt
Read of options file complete.

2 row/cols, 2 non-zeros, 50.00% dense.

Path 4.7.03 (Thu Jan 24 15:44:12 2013)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


(:Solved, [0.0, 0.0], [0.0, 0.0])

In [76]:
# phi = x - 1
# phi >= 0 \perp lambda >= 0
# z = [x, lambda]
# f = [0, x - 1]
# lb = [-Inf, 0]
# ub = [Inf, Inf]

lb = [-Inf, 0]
ub = [Inf, Inf]
f = z -> [0, z[1] - 1]
options(output=:no)
status, z, F = solveLCP(f, lb, ub)

Reading options file path.opt
Read of options file complete.

2 row/cols, 1 non-zeros, 25.00% dense.

Path 4.7.03 (Thu Jan 24 15:44:12 2013)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


(:Solved, [1.0, 0.0], [0.0, 0.0])

In [77]:
# phi = x - y
# phi >= 0 \perp lambda >= 0
# 0.5 <= y <= 1
# z = [x, y, lambda]

f = z -> [0., 0., z[1] - z[2]]
lb = [-Inf, 0.5, 0]
ub = [Inf, 1.0, Inf]
options(output=:no)
status, z, F = solveLCP(f, lb, ub)

Reading options file path.opt
Read of options file complete.

3 row/cols, 2 non-zeros, 22.22% dense.

Path 4.7.03 (Thu Jan 24 15:44:12 2013)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


(:Solved, [0.5, 0.5, 0.0], [0.0, 0.0, 0.0])

In [78]:
# phi = x - y
# phi >= 0 \perp lambda >= 0
# 0.5 <= y <= 1
# 2x - y >= 10

# convert into:
# phi = x - y
# phi >= 0 \perp lambda >= 0
# 0.5 <= y <= 1
# s1 >= 0
# -2x + y + s1 == -10

# z = [x, y, s1, s2, lambda]
# f = [0, 0, 0, -2x + y + s1 - 10, x - y]

f = z -> [0., 0., 0., -2z[1] + z[2] + z[3] + 10, z[1] - z[2]]
lb = [-Inf, 0.5, 0., -Inf, 0]
ub = [Inf, 1.0, Inf, Inf, Inf]
options(output=:no)
status, z, F = solveLCP(f, lb, ub)

Reading options file path.opt
Read of options file complete.

5 row/cols, 5 non-zeros, 20.00% dense.

Path 4.7.03 (Thu Jan 24 15:44:12 2013)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

 ** EXIT - solution found.


(:Solved, [5.25, 0.5, 0.0, -1.0e20, 0.0], [0.0, 0.0, 0.0, 0.0, 4.75])

In [79]:
using JuMP

In [80]:
m = Model(solver=PATHInterface.PATHLCPSolver())
@variable m x
@variable m 0.5 <= y <= 1
@constraint m 2x - y >= 10
m
solve(m)

Could not open options file: path.opt
Using defaults.
4 row/cols, 3 non-zeros, 18.75% dense.

Path 4.7.03 (Thu Jan 24 15:44:12 2013)
Written by Todd Munson, Steven Dirkse, and Michael Ferris

INITIAL POINT STATISTICS
Zero column of order. . . . . .  0.0000e+00 var: (x[    4])
Zero row of order . . . . . . .  0.0000e+00 eqn: (f[    1])
Zero row of order . . . . . . .  0.0000e+00 eqn: (f[    2])
Zero row of order . . . . . . .  0.0000e+00 eqn: (f[    3])
Total zero columns. . . . . . .  1
Total zero rows . . . . . . . .  3
Maximum of X. . . . . . . . . .  1.0000e+20 var: (x[    1])
Maximum of F. . . . . . . . . .  2.0000e+20 eqn: (f[    4])
Maximum of Grad F . . . . . . .  2.0000e+00 eqn: (f[    4])
                                            var: (x[    1])

INITIAL JACOBIAN NORM STATISTICS
Maximum Row Norm. . . . . . . .  4.0000e+00 eqn: (f[    4])
Minimum Row Norm. . . . . . . .  4.0000e+00 eqn: (f[    4])
Maximum Column Norm . . . . . .  2.0000e+00 var: (x[    1])
Minimum Column Norm

:Optimal

In [81]:
getvalue(x), getvalue(y)

(5.25, 0.5)